In [2]:
import numpy as np
import scipy.signal as sig
import scipy.io
import os
import librosa
import sys
from IPython.display import display, Audio
import time
import pickle

import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('font', size=10)
plt.rc('font', family='serif')
plt.rc('lines', linewidth = .5)
cm = 2.54
# setting path
sys.path.append("../")
from Utils.dB import *
from HN_decomposition.Subspace.ESPRIT_numpy import *
from HN_decomposition.Subspace.PQMF import *

def RMS(x):
        return np.std(x)

figsize = (8.5/cm, 6/cm)

def normalize(input_array, value = 1):
        return input_array/np.amax(np.abs(input_array))*value

NameError: name 'npt' is not defined

In [ ]:
## Base params

dpi = 300
cmap = 'gray_r'

## ESTER Criterion

In [ ]:
amps_nus = [[1, .1], [1, .2], [1, .4], [.9, .04]]

amp_noise = .1
length_smp = 512

x = np.zeros(length_smp, dtype= np.complex128)

for amp_nu in amps_nus:
    amp = amp_nu[0]
    nu = amp_nu[1]
    x += amp*np.sin(2*np.pi*nu*np.arange(length_smp))

x += amp_noise*np.random.randn(length_smp)


_, J_list = ESTER(
    x=x,
    start_idx = 15,
    window_length = 256,
    max_poles = 30,
    factor = 10
)

plt.stem(J_list)
plt.show()

## Loading the audio file

In [ ]:
start_sec = .82
length_sec = 4

start_sec = 0
length_sec = 5

fileName = os.path.join('..', '..', 'Mesures', 'Mesures_2023.01.27_LAM', 'Medium_1', 'Corde_Acier', 'C_3_P_3_1.mat')
print(fileName)
data = scipy.io.loadmat(fileName)
print(data.keys())

sr_hz_old =  data['fs'][0,0]
x = data['acc_t'][int(sr_hz_old*start_sec):int(sr_hz_old*(start_sec+length_sec)),0].T
t = np.arange(len(x))/sr_hz_old

print(f'sample rate : {sr_hz_old} Hz')

plt.figure(figsize = figsize)
plt.plot(t,x,'k', linewidth = .5)
plt.show()

plt.figure(figsize = figsize)
plt.plot(np.fft.rfftfreq(len(x), 1/sr_hz_old), dB20(np.fft.rfft(x)))
#plt.semilogx()
plt.xlim(0, sr_hz_old/2)
plt.show()

## Resampling

In [ ]:
sr_hz = 22050/2
x = librosa.resample(x, orig_sr = sr_hz_old, target_sr = sr_hz)
t = np.arange(len(x))/sr_hz
plt.figure(figsize = figsize)
plt.plot(t,x)
plt.show()
plt.figure(figsize = figsize)
plt.plot(np.fft.rfftfreq(len(x), 1/sr_hz), dB20(np.fft.rfft(x)))
plt.xlim(0, sr_hz/2)
plt.show()

In [ ]:
save_folder = os.path.join("..", "..", "PAM_overleaf", "Notes", "HR", "Partials_Tracking", "Figures")

## Harmonic+Noise Decomposition

In [ ]:
start_idx = np.argmax(np.abs(sig.hilbert(x)))
factor = 10

## Computations
xWhitened, ARFilter = whiten_signal(
    x = x,
    n_fft = 512,
    rankFilter_bins = 200,
    rankFilter_rank = .3,
    ARFilter_length = 18,
)

num_poles, J_list = ESTER(
    x = xWhitened,
    start_idx=start_idx,
    window_length=512,
    max_poles = 50,
    factor = factor
)


print(num_poles)
plt.figure(figsize=figsize)
markerline, stemline, baseline = plt.stem(J_list)
plt.setp(markerline, markersize = 3)
plt.axhline(np.amax(J_list)/factor)
plt.xlabel('Nombre de pôles')
plt.ylabel('Critère')
plt.tight_layout()
plt.savefig('ESTER.pdf')
plt.show()

## Filter Bank Design

In [ ]:

num_bands = 16
num_taps = num_bands*8 - (1*(num_bands%2==0))
tol_transitionBand = .2/num_bands

x_test = np.zeros(2048)
x_test[0] = 1

Analysis_filters, Synthesis_filters = nearPR_CMFB(
    num_bands = num_bands,
    num_taps = num_taps,
    tol_transitionBand=tol_transitionBand)
## Plotting
save_folder = os.path.join(
    "..", "..", "PAM_overleaf", "Notes", "HR", "Preprocessing", "Figures")
n_fft = 2048
xRecons = np.zeros(len(x_test))

for band_idx in range(num_bands):
    xFilter = sig.lfilter(Analysis_filters[band_idx], [1], x_test)
    xInsert = np.zeros(len(x_test))
    xInsert[::num_bands] = xFilter[::num_bands]*num_bands
    xRecons+=sig.lfilter(Synthesis_filters[band_idx], [1], xInsert)


f = np.fft.rfftfreq(n_fft, 1)
plt.figure(figsize = figsize)
for band_idx in range(num_bands):
    plt.plot(f, dB20(np.fft.rfft(Analysis_filters[band_idx], n = n_fft)), '--')
plt.plot(f, dB20(np.fft.rfft(xRecons)), 'k', linewidth = 2)
plt.xlim(0,.5)
plt.ylim(-5, 5)
plt.ylabel('Amplitude [dB]')
plt.xlabel("Fréquence réduite")
plt.tight_layout()
plt.savefig(
    fname = os.path.join(save_folder, "filterBank.pdf"),
    bbox_inches = 'tight'
)
plt.show()

xRecons = np.zeros(len(x))
for band_idx in range(num_bands):
    xFilter = sig.lfilter(Analysis_filters[band_idx], [1], x)
    xInsert = np.zeros(len(x))
    xInsert[::num_bands] = xFilter[::num_bands]*num_bands
    xRecons+=sig.lfilter(Synthesis_filters[band_idx], [1], xInsert)

plt.plot(dB20(np.fft.rfft(x)), linewidth = 2)
plt.plot(dB20(np.fft.rfft(xRecons)))
plt.show()
display(Audio(data = x, rate = sr_hz))
display(Audio(data = xRecons, rate = sr_hz))


In [ ]:
##Parameters
window_length = 32
hop_length = 8
window_type = 'hann'
ester_factor = 50

num_bands = 16

## Initializing output vectors

## Computations
tic = time.time()
xHarmo, xNoise = multiband_HN(
    x,
    tracking_method = 'FAPI',
    num_bands = num_bands,
    ester_factor=ester_factor,
    window_length=window_length,
    hop_length=hop_length,
    FAPI_beta = .9)

toc = time.time()

print(f'Processing time : {round(toc-tic, 2)}s')

display(Audio(data = x, rate = sr_hz))
display(Audio(data = np.real(xHarmo), rate = sr_hz))
display(Audio(data = np.real(xNoise), rate = sr_hz))

fig, ax = plt.subplots(nrows= 1, ncols = 3, sharey = True, figsize = (15/cm, 7/cm))
save_folder = os.path.join("HN", "Subspace", "Audio")
cmap = 'inferno'

f, t, x_stft = sig.stft(np.real(x), fs = sr_hz, return_onesided=True, scaling = 'psd')
ax[0].pcolormesh(t, f, dB10(x_stft), cmap = cmap, rasterized = True)
ax[0].set_title("Signal")
ax[0].set_ylabel("Fréquence [Hz]")
ax[0].set_xlabel("Temps [s]")
f, t, x_stft = sig.stft(np.real(xHarmo), fs = sr_hz, return_onesided=True, scaling = 'psd')
ax[1].pcolormesh(t, f, dB10(x_stft), cmap = cmap, rasterized = True)
ax[1].set_title("Composantes\nharmoniques")
ax[1].set_xlabel("Temps [s]")
f, t, x_stft = sig.stft(np.real(xNoise), fs = sr_hz, return_onesided=True, scaling = 'psd')
ax[2].pcolormesh(t, f, dB10(x_stft), cmap = cmap, rasterized = True)
ax[2].set_title("Bruit")
ax[2].set_xlabel("Temps [s]")
plt.tight_layout()

plt.savefig(os.path.join(save_folder, 'spectrogrammesHN.pdf'))
plt.show()

save_dict = {
    "input" : x,
    "harmonic" : xHarmo,
    "noise" : xNoise,
    "sampleRate" : sr_hz
}

with open("test_HN.pickle", "wb") as file:
    pickle.dump(save_dict, file, pickle.HIGHEST_PROTOCOL)

print(sr_hz)
scipy.io.wavfile.write(filename = os.path.join(save_folder, "input.wav"), rate = int(sr_hz), data = normalize(x))
scipy.io.wavfile.write(filename = os.path.join(save_folder, "input.wav"), rate = int(sr_hz), data = normalize(np.real(xHarmo)))
scipy.io.wavfile.write(filename = os.path.join(save_folder, "input.wav"), rate = int(sr_hz), data = normalize(np.real(xNoise)))